In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'tmnist-alphabet-94-characters:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1564532%2F2830968%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240409%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240409T182841Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dd8ca16ade07ec9d295ea309a8ae60f201b97187be5d48f6b5193d087c7c6a27c1d85af811fa153e614729985b11c770d05bce3d4f6fd0b2a2abd97446e5a293ec608fc3ffc0dc62d4acc579fd73a70b97d5264a8e15a16d51bed82aa5fb352427297e6f8c604dbb5be90a8342b31026280a9c4d2ceb8bc3cb0feccd31996d771495eb8204395282d3650851d9ff96b81ac7422c43ed17749bee03c8e25b65e1b6001cf97b2f84450b2c0df63df813ebc8843a570c33ea35f03a485a04fa34f80a04e9dc26d18f8584258c786857ca51ed67135f54339ba71fb36dafae6a1d1249e94d2977b15cafae3bba2d2d0c1804ec558b39d3b82f65969165cdc39005c7e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 88850214 bytes downloaded
Downloaded and uncompressed: tmnist-alphabet-94-characters
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmnist-alphabet-94-characters/94_character_TMNIST.csv


In [4]:
pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 MB 4.2 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:



# Load and preprocess data
df = pd.read_csv("/kaggle/input/tmnist-alphabet-94-characters/94_character_TMNIST.csv")
X = df.drop(['names', 'labels'], axis=1).values.reshape(df.shape[0], 28, 28, 1) / 255
y = df['labels']

chars = sorted(list(df['labels'].unique()))
vocab_size = len(chars)

itos = {i: ch for i, ch in enumerate(chars)}
stoi = {ch: i for i, ch in enumerate(chars)}

y_upd = np.array([stoi[ch] for ch in y])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_upd, test_size=0.2, random_state=1337)

# Convert labels to one-hot encoding
y_train_one_hot = to_categorical(y_train, num_classes=vocab_size)
y_test_one_hot = to_categorical(y_test, num_classes=vocab_size)

# Define the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128),
    BatchNormalization(),
    ReLU(),
    Dense(vocab_size, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=3e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_one_hot, batch_size=128, epochs=10, validation_data=(X_test, y_test_one_hot))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/10
 824/1714 [=============>................] - ETA: 3:46 - loss: 0.8351 - accuracy: 0.8444

In [ ]:
import h2o
from h2o.automl import H2OAutoML

# Initialize H2O cluster
h2o.init()

# Convert Pandas DataFrame to H2OFrame
h2o_df = h2o.H2OFrame(df)

# Define the target column as categorical
h2o_df['labels'] = h2o_df['labels'].asfactor()

# Split the data
train, test = h2o_df.split_frame(ratios=[0.8], seed=1337)

# Define predictor columns
predictors = h2o_df.columns[:-1]

In [ ]:
# Run AutoML with only the neural network algorithm
aml = H2OAutoML(max_runtime_secs=60, exclude_algos=["DRF", "GLM", "XGBoost", "GBM", "StackedEnsemble"])
aml.train(x=predictors, y='labels', training_frame=train)

# View the AutoML leaderboard
lb = aml.leaderboard
print(lb)



In [ ]:
# Get the best model from AutoML
best_model = aml.leader

# Evaluate the best model on the test set
perf = best_model.model_performance(test)
print(perf)


In [ ]:
# Get the best model from AutoML
best_model = aml.leader

# Make predictions on the test set
predictions = best_model.predict(test)

# Calculate accuracy
accuracy = (predictions['predict'] == test['labels']).mean()
print(f"Test accuracy: {accuracy * 100:.2f}%")
